<a href="https://colab.research.google.com/github/Jushef/Azure-AutoML/blob/main/MMSA/01_Data_Preparation_My_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import datetime as dt
from azureml.core import Workspace
from scripts.helper import split_data

In [ ]:
ManyModelsSample = pd.read_csv('ManyModelsSampleData.csv', header = 0) 

In [ ]:
target_path = 'MMSA_Sample_Folder' 
os.makedirs(target_path, exist_ok=True)

In [ ]:
ManyModelsSample

In [ ]:
ManyModelsSample['Date'] = ManyModelsSample['Date'].apply(lambda x: dt.datetime.strptime(x, '%m/%d/%Y'))

In [ ]:
for x, y in ManyModelsSample.groupby('Store'):
    y.to_csv('MMSA_Sample_Folder/{}.csv'.format(x), header=True, index_label=False)

## 2.0 Split data in two sets

Ahora dividiremos cada conjunto de datos en dos partes: una se utilizará para la formación y la otra para la simulación de la previsión por lotes. Los archivos de formación contendrán los registros de datos anteriores a "1992-5-28" y la última parte de cada serie se almacenará en los archivos de inferencia.


In [ ]:
# Proporcione el nombre de la columna de la marca de tiempo en los datos y la fecha a partir de la cual se va a dividir en el conjunto de datos de inferencia
timestamp_column = 'Date'
split_date = '2021-03-01'

# Dividir cada archivo y almacenarlo en el directorio correspondiente
train_path, inference_path = split_data(target_path, timestamp_column, split_date)

## 3.0 Upload data to Datastore in AML Workspace


In [ ]:
ws = Workspace.from_config()

In [ ]:
#  Conectar con el almacén de datos por defecto
datastore = ws.get_default_datastore()

# Cargar los datos del entrenamiento
ds_train_path = target_path + '_train'
datastore.upload(src_dir=train_path, target_path=ds_train_path, overwrite=True)

# Cargar datos de inferencia
ds_inference_path = target_path + '_inference'
datastore.upload(src_dir=inference_path, target_path=ds_inference_path, overwrite=True)

In [ ]:
from azureml.core.dataset import Dataset

# Crear conjuntos de datos de archivos
ds_train = Dataset.File.from_files(path=datastore.path(ds_train_path), validate=False)
ds_inference = Dataset.File.from_files(path=datastore.path(ds_inference_path), validate=False)

# Registrar los conjuntos de datos de los archivos
dataset_name = 'MMSA_Sample'
train_dataset_name = dataset_name + '_train'
inference_dataset_name = dataset_name + '_inference'
ds_train.register(ws, train_dataset_name, create_new_version=True)
ds_inference.register(ws, inference_dataset_name, create_new_version=True)